En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [9]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [10]:
%load_ext line_profiler
%load_ext memory_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# Pregunta 1

Se tomaron dos enfoques distintos. El primero consistió en leer los datos con la librería JSON y luego trabajar la información con Pandas. Para el segundo enfoque se utilizó PySpark para la lectura y tratamiento de los datos.

En cuanto al tiempo de ejecución con el primer método, se observó que los tiempos eran estables y repetibles, con una duración de alrededor de 11 segundos. En cambio, el segundo método dependía de forma sustancial del estado de la sesión de Spark. En la primera ejecución consecutiva, los tiempos de ejecución rondaban los 15 segundos, muy distintos a los tiempos de las ejecuciones posteriores, que rondaban entre 5 y 6 segundos.

Debido a esto, se tomó la decisión de elegir el primer método para q1_time, únicamente por su estabilidad. Cabe considerar que, en el caso de tener un ambiente de ejecución más estable, el segundo método podría ofrecer mejores resultados a largo plazo.

En cuanto al uso de memoria, este fue un análisis más directo. Con el método 1, siempre se observó un uso de hasta 1300 MiB, muy por encima del método 2, que rondaba los 300 MiB de forma constante. Por ende, se seleccionó el método 2 para q1_memory

## Metodo 1

Se ejecuto multiples veces y siempre se consigui un tiempo de ejecucion de alrededor de 11s

In [8]:
from q1_time import  q1_time

In [ ]:
%lprun -f q1_time q1_time(file_path=file_path)

In [ ]:
%lprun -f q1_time q1_time(file_path=file_path)

In [ ]:
%mprun -f q1_time q1_time(file_path=file_path)

Mem pregunta 1

In [3]:
from q1_memory import  q1_memory

In [ ]:
%lprun -f q1_memory q1_memory(file_path=file_path)

In [ ]:
%lprun -f q1_memory q1_memory(file_path=file_path)

In [ ]:
%mprun -f q1_memory q1_memory(file_path=file_path)

# Pregunta 2

Para este problema se tomó un enfoque similar al de la pregunta anterior, pero a diferencia de esta, para el tiempo se utilizó PySpark y para la memoria, funciones convencionales.

Dado que solo es necesario contar ocurrencias a nivel de registro, se puede iterar fila por fila.

Lamentablemente, este enfoque no obtuvo los resultados esperados, y la metodología sin PySpark utilizó prácticamente la misma cantidad de memoria que la que usaba PySpark, e incluso más en algunas pruebas.

In [5]:
from q2_time import  q2_time

%lprun -f q2_time q2_time(file_path=file_path)

your 131072x1 screen size is bogus. expect trouble
24/10/14 22:54:59 WARN Utils: Your hostname, pescara-pc resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/14 22:54:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 22:55:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Timer unit: 1e-09 s

Total time: 18.4042 s
File: /home/sandbox/files/latam-challenge/src/q2_time.py
Function: q2_time at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45         1 3905414685.0    4e+09     21.2      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                           
    47         1 5956659988.0    6e+09     32.4      data = spark.read.json(file_path)
    48         1  335665521.0    3e+08      1.8      dataframe = get_most_used_emojis(spark, data)
    49                                           
    50         1       1793.0   1793.0      0.0      del data
    51                                           
    52         2      27091.0  13545.5      0.0      output = [
    53         1 7216616653.0    7e+09     39.2          tuple(row) for row in dataframe.select([col("emoji"), col("count")]).co

In [7]:
%mprun -f q2_time q2_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    44    174.9 MiB    174.9 MiB           1   def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45    174.9 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                         
    47    174.9 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    48    174.9 MiB      0.0 MiB           1       dataframe = get_most_used_emojis(spark, data)
    49                                         
    50    174.9 MiB      0.0 MiB           1       del data
    51                                         
    52    174.9 MiB      0.0 MiB          24       output = [
    53    174.9 MiB      0.0 MiB          11           tuple(row) for row in dataframe.select([col("emoji"), col("count")]).collect()
    54                                             ]
    55                      

In [8]:
from q2_memory import  q2_memory

%lprun -f q2_memory q2_memory(file_path)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-09 s

Total time: 9.83701 s
File: /home/sandbox/files/latam-challenge/src/q2_memory.py
Function: q2_memory at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     8         1        912.0    912.0      0.0      result = dict()
     9         2     451684.0 225842.0      0.0      with open(file_path, "r") as f:
    10                                                   # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    11      3080   55925861.0  18157.7      0.6          for line in f:
    12      3080 8193672422.0    3e+06     83.3              data = pd.json_normalize(json.loads(line))
    13      6158  311252272.0  50544.4      3.2              for c in data.content.to_list():
    14      4139 1274725765.0 307979.2     13.0                  for e in emoji.emoji_list(c):
    15      1060     983372.0    927.7      0.0    

In [ ]:
%mprun -f q2_memory q2_memory(file_path)

# Pregunta 3

Problemática similar al ejercicio anterior, salvo por la diferencia de que el dato venía dentro de una lista de JSONs y no en un texto plano (exceptuando el hecho de que se tenían que extraer emojis).

Por esta razón, se tomó un enfoque similar al de la pregunta anterior, con resultados similares.

In [11]:
from q3_time import  q3_time

%lprun -f q3_time q3_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 2.58777 s
File: /home/sandbox/files/latam-challenge/src/q3_time.py
Function: q3_time at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def q3_time(file_path: str) -> List[Tuple[str, int]]:
    26         1   96438348.0    1e+08      3.7      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    27                                           
    28         1  669796235.0    7e+08     25.9      data = spark.read.json(file_path)
    29         1   52012522.0    5e+07      2.0      dataframe = get_most_mentions(spark, data)
    30                                           
    31         1        471.0    471.0      0.0      del data
    32                                           
    33         2      10800.0   5400.0      0.0      output = [
    34         1 1319767472.0    1e+09     51.0          tuple(row) for row in dataframe.select([col("username"), col("count")]).co

In [13]:
%mprun -f q3_time q3_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q3_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    25    177.3 MiB    177.3 MiB           1   def q3_time(file_path: str) -> List[Tuple[str, int]]:
    26    177.3 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    27                                         
    28    177.3 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    29    177.3 MiB      0.0 MiB           1       dataframe = get_most_mentions(spark, data)
    30                                         
    31    177.3 MiB      0.0 MiB           1       del data
    32                                         
    33    177.3 MiB      0.0 MiB          24       output = [
    34    177.3 MiB      0.0 MiB          11           tuple(row) for row in dataframe.select([col("username"), col("count")]).collect()
    35                                             ]
    36                      

In [14]:
from q3_memory import  q3_memory

%lprun -f q3_memory q3_memory(file_path)

Timer unit: 1e-09 s

Total time: 310.855 s
File: /home/sandbox/files/latam-challenge/src/q3_memory.py
Function: q3_memory at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     7         1       1473.0   1473.0      0.0      result = dict()
     8         2      48370.0  24185.0      0.0      with open(file_path, "r") as f:
     9                                                   # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    10    117408 1857681054.0  15822.4      0.6          for line in f:
    11    117407        3e+11    3e+06     97.2              data = pd.json_normalize(json.loads(line))
    12    117407 3157675293.0  26895.1      1.0              if "quotedTweet.mentionedUsers" in data.columns: # Hay registros que no tienen la columna
    13     41436 3320505758.0  80135.8      1.1                  if data["quotedTweet.men

In [15]:
%mprun -f q3_memory q3_memory(file_path)

Filename: /home/sandbox/files/latam-challenge/src/q3_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    179.5 MiB    179.5 MiB           1   def q3_memory(file_path: str) -> List[Tuple[str, int]]:
     7    179.5 MiB      0.0 MiB           1       result = dict()
     8    185.5 MiB      0.0 MiB           2       with open(file_path, "r") as f:
     9                                                 # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    10    185.5 MiB      0.1 MiB      117408           for line in f:
    11    185.5 MiB      5.8 MiB      117407               data = pd.json_normalize(json.loads(line))
    12    185.5 MiB      0.1 MiB      117407               if "quotedTweet.mentionedUsers" in data.columns: # Hay registros que no tienen la columna
    13    185.5 MiB      0.1 MiB       41436                   if data["quotedTweet.mentionedUsers"][0]: # Y otros que vienen con "None"
    14    185.5 MiB      0.0 MiB    